In [1]:
# 31000/4

In [5]:
!gpustat

2a974e97ed31              Sun Jun  8 06:47:27 2025  535.216.01
[0] NVIDIA A100-SXM4-40GB | 25'C,   0 % |   628 / 40960 MB |
[1] NVIDIA A100-SXM4-40GB | 25'C,   0 % |   626 / 40960 MB |
[2] NVIDIA A100-SXM4-40GB | 24'C,   0 % |   628 / 40960 MB |
[3] NVIDIA A100-SXM4-40GB | 31'C,  72 % | 25103 / 40960 MB |


In [8]:
# 7750.0 * 100


In [5]:
# 775000/5000 

In [6]:
# 155*3/24

In [7]:
# import pandas as pd
# import numpy as np
# import os
# !pip install nibabel

In [8]:
# csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/pyradiomics_sybil_demog_CV.csv"

In [9]:
# df = pd.read_csv(csv_path)
# print(df.shape)
# print(df.columns[:4],df.columns[-10:])
# df.head(2)

In [10]:
# df[df["MRN"]=="1093523"]

# This code has a bug, if the date starts with 0, it will be missed

In [15]:
root_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/"
saving_path = "/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/"
import os
import nibabel as nib
import numpy as np
import pandas as pd
csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/pyradiomics_sybil_demog_CV.csv"
df = pd.read_csv(csv_path)
def which_address_exist(add1, add2):
    if os.path.exists(add1):
        return add1
    elif os.path.exists(add2):
        return add2
    else:
        return None

def get_base_name(filename):
    """Remove .nii.gz or .nii from filename cleanly."""
    if filename.endswith(".nii.gz"):
        return filename[:-7]
    elif filename.endswith(".nii"):
        return filename[:-4]
    else:
        return os.path.splitext(filename)[0]

#missing_MRNs = ['477516', '730556', '737729', '281227', '333062', '297791A', '422865', '2338015B', '754274', '370975']
for mrn in set(df["MRN"].to_list()):
  
    root_mrn = os.path.join(root_path, mrn+"_D")
    print(root_mrn)
    if os.path.exists(root_mrn):
        dates = df[df["MRN"]==mrn ]["date"].to_list()
        dates = [str(date) for date in dates]
        #print(dates)
        for date in dates:
            add1 = os.path.join(root_mrn, date)
            add2 = os.path.join(root_mrn, date+"_")
            root_mrn_date = which_address_exist(add1, add2)
            if root_mrn_date is None:
                continue
            df_mrn = df[(df["MRN"] == mrn) & (df["date"] == int(date))]
            info = df_mrn[['date', 'Date_To_Cancer', '1_year_risk', '2_year_risk', '3_year_risk', '4_year_risk','5_year_risk', '6_year_risk', 'age at dx', 'Sex', 'Race','Tobacco Use']]
            info_dict = info.iloc[0].to_dict()
            #print(df_mrn)
            #print(info)
            #print(info_dict)
            #print(mrn, root_mrn_date)
            ct_ = [i for i in os.listdir(root_mrn_date) if i.endswith(".nii.gz") and not any(x in i.lower() for x in ["_p", "seg", "sybil"])]
            seg_ = [i for i in os.listdir(root_mrn_date) if i.endswith(".nii.gz") and "_p" in i.lower()]
            if len(ct_)!=1 and len(seg_)!=1:
                print(len(ct_), len(seg_))
                continue
            ct = os.path.join(root_mrn_date, ct_[0])
            seg = os.path.join(root_mrn_date, seg_[0])
            #print(ct)
            #print(seg)
            ct_w = nib.load(ct)
            seg_w = nib.load(ct)
            # Check number of slices (assumes axial slices along third dimension)
            if ct_w.shape != seg_w.shape:
                print("Shape mismatch:", ct_w.shape, seg_w.shape, "→ Skipping", mrn, date)
                continue

            spacing_ct = ct_w.header.get_zooms()
            spacing_seg = seg_w.header.get_zooms()

            base_name = get_base_name(ct_[0])
            # Save data
            out_path = os.path.join(saving_path, base_name + ".npz")
            print("saving in ", out_path)
            np.savez(
                out_path,
                spacing_ct=spacing_ct,
                ct=ct_w.get_fdata(),  # native dtype
                seg=seg_w.get_fdata(),  # native dtype
                **info_dict
            )
        print("Saved:", out_path)
        print()

/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/477516_D


NameError: name 'out_path' is not defined

In [4]:
import numpy as np
a = np.load("/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/420022_160315__3.11__.nii.npz")

In [6]:
a = np.load("/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/420022_160315__3.11__.npz")

In [7]:
for i in a.keys():
    print(i)

spacing_ct
ct
seg
date
Date_To_Cancer
1_year_risk
2_year_risk
3_year_risk
4_year_risk
5_year_risk
6_year_risk
age at dx
Sex
Race
Tobacco Use


In [9]:
a["ct"].shape

(512, 512, 140)

In [11]:
a["seg"].shape

(512, 512, 140)

In [12]:
a["ct"].dtype

dtype('float64')

In [13]:
a["seg"].dtype

dtype('float64')

In [1]:
# import numpy as np
# spacing_ct = np.array([0.7, 0.7, 2.0])
# ct = np.random.randint(-1000, 200, (512, 512, 140), dtype=np.int16)
# seg = np.zeros((512, 512, 140), dtype=np.uint8)
# seg[200:220, 200:220, 60:65] = 1  # small cuboid region set to 1
# age = 63
# Sex = "Male"
# Race = "Asian"
# histology_label = 1  # ← squamous
# histology_adc = 0
# histology_squ = 1
# histology_smc = 0
# np.savez("synthetic_lung_case.npz",
#          spacing_ct=spacing_ct,
#          ct=ct,
#          seg=seg,
#          age=age,
#          Sex=Sex,
#          Race=Race,
#          histology_label=histology_label,
#          histology_adc=histology_adc,
#          histology_squ=histology_squ,
#          histology_smc=histology_smc)

In [10]:
# data = np.load("synthetic_lung_case.npz")

# # Access elements
# ct = data["ct"]
# seg = data["seg"]
# spacing_ct = data["spacing_ct"]
# age = int(data["age"])
# Sex = str(data["Sex"])
# Race = str(data["Race"])

# print("CT shape:", ct.shape)
# print("Seg shape:", seg.shape, "Unique labels:", np.unique(seg))
# print("Spacing:", spacing_ct)
# print("Age:", age)
# print("Sex:", Sex)
# print("Race:", Race)


CT shape: (512, 512, 140)
Seg shape: (512, 512, 140) Unique labels: [0 1]
Spacing: [0.7 0.7 2. ]
Age: 63
Sex: Male
Race: Asian


In [2]:
root_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/"
saving_path = "/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/"
import os
import nibabel as nib
import numpy as np
import pandas as pd
csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/pyradiomics_sybil_demog_CV.csv"
df = pd.read_csv(csv_path)

In [7]:
len(list(set(df["MRN"].to_list())))
len(list(set([i.split("_")[0] for i in os.listdir(saving_path)])))

122

In [23]:
# 1) all MRNs in your DataFrame
mrn_set = set(df["MRN"].to_list())

# 2) all MRNs as taken from the filenames (prefix before "_")
file_mrn_set = set(i.split("_")[0] for i in os.listdir(saving_path))

# (a) MRNs present in df but missing on disk:
missing_on_disk = mrn_set - file_mrn_set
print(f"{len(missing_on_disk)} MRN(s) in df not found in filenames:")
print(missing_on_disk)

2 MRN(s) in df not found in filenames:
{'297791A', '2338015B'}


In [26]:
df

,Unnamed: 0,MRN,date,Date_To_Cancer,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,...,1_year_risk,2_year_risk,3_year_risk,4_year_risk,5_year_risk,6_year_risk,age at dx,Sex,Race,Tobacco Use
0,171,2147359,170222,0.00,0.818949,0.635424,10.010605,15.754225,18.292267,17.721337,...,0.021629,0.038573,0.071919,0.079270,0.095846,0.135681,75.989041,M,Asian NOS,Former
1,172,2147359,160309,0.96,0.957088,0.469376,6.676237,14.223633,16.563390,16.781006,...,0.021629,0.038573,0.071919,0.079270,0.095846,0.135681,75.989041,M,Asian NOS,Former
2,170,2147359,161006,0.38,0.812699,0.466205,8.742538,18.752558,20.767153,19.788782,...,0.028122,0.058385,0.084379,0.096455,0.105678,0.150757,75.989041,M,Asian NOS,Former
3,58,2229583,170428,0.51,0.619535,0.527080,11.811456,22.409217,22.841933,25.943905,...,0.017994,0.034271,0.060600,0.069836,0.081818,0.125812,69.728767,F,Asian NOS,Never (<100 lifetime cigarettes)
4,59,2229583,171101,0.00,0.695085,0.506186,12.781377,25.250358,27.158481,29.406367,...,0.017994,0.029688,0.052035,0.069836,0.081818,0.125812,69.728767,F,Asian NOS,Never (<100 lifetime cigarettes)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,402,900183,110811,5.31,0.654120,0.567661,9.712776,17.110174,15.584518,18.419615,...,0.021629,0.038573,0.071919,0.079270,0.095846,0.135681,67.790000,F,Other,Never (<100 lifetime cigarettes)
684,403,900183,120517,4.54,0.517663,0.368625,8.281930,22.467092,15.283425,21.569436,...,0.028122,0.050187,0.071919,0.079270,0.095846,0.135681,67.790000,F,Other,Never (<100 lifetime cigarettes)
685,405,900183,150522,1.53,0.681813,0.462026,12.084474,26.155378,22.947482,28.627883,...,0.021629,0.045262,0.071919,0.079270,0.095846,0.135681,67.790000,F,Other,Never (<100 lifetime cigarettes)
686,395,900183,141111,2.06,0.749123,0.430171,10.595525,24.630975,22.947482,26.562500,...,0.028122,0.054802,0.080800,0.089659,0.100888,0.146625,67.790000,F,Other,Never (<100 lifetime cigarettes)


In [20]:
root_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/"
saving_path = "/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/"
import os
import nibabel as nib
import numpy as np
import pandas as pd
csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/pyradiomics_sybil_demog_CV.csv"
csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/all_pyradiomics_sybil_demog_not_all.csv"
df = pd.read_csv(csv_path)
def which_address_exist(add1, add2):
    if os.path.exists(add1):
        return add1
    elif os.path.exists(add2):
        return add2
    else:
        return None

def get_base_name(filename):
    """Remove .nii.gz or .nii from filename cleanly."""
    if filename.endswith(".nii.gz"):
        return filename[:-7]
    elif filename.endswith(".nii"):
        return filename[:-4]
    else:
        return os.path.splitext(filename)[0]

missing_MRNs = ['477516', '730556', '737729', '281227', '333062', '297791A', '422865', '2338015B', '754274', '370975']
missing_MRNs = ["1094315"]
for mrn in missing_MRNs: 
# for mrn in set(df["MRN"].to_list()):

    root_mrn = os.path.join(root_path, mrn+"_D")
    print(root_mrn)
    if os.path.exists(root_mrn):
        dates = df[df["MRN"]==mrn ]["date"].to_list()
        #print(dates)
        dates = [str(date) for date in dates]
        #dates = ["141125"]
        print(dates)
        for date in dates:

            # The csv is wrong showing 030623 as 30623 !!!!
            #print(len(date), date, date*2)#date is str
            if len(date) == 6:
                pass
                #continue
            elif len(date) == 5:
                date = "0"+date
                continue
            elif len(date) == 4:
                date = "00"+date
            add1 = os.path.join(root_mrn, date)
            add2 = os.path.join(root_mrn, date+"_")

            root_mrn_date = which_address_exist(add1, add2)
            print("root_mrn_date: ", root_mrn_date)
            if root_mrn_date is None:
                continue
            df_mrn = df[(df["MRN"] == mrn) & (df["date"] == int(date))]
            info = df_mrn[['date', 'Date_To_Cancer', '1_year_risk', '2_year_risk', '3_year_risk', '4_year_risk','5_year_risk', '6_year_risk', 'age at dx', 'Sex', 'Race','Tobacco Use']]
            #info = df_mrn[['date', 'Date_To_Cancer', '1_year_risk', '2_year_risk', '3_year_risk', '4_year_risk','5_year_risk', '6_year_risk', 'Sex', 'Race','Tobacco Use']]

            info_dict = info.iloc[0].to_dict()
            #print(df_mrn)
            #print(info)
            #print(info_dict)
            #print(mrn, root_mrn_date)
            ct_ = [i for i in os.listdir(root_mrn_date) if i.endswith(".nii.gz") and not any(x in i.lower() for x in ["_p", "seg", "sybil"])]
            seg_ = [i for i in os.listdir(root_mrn_date) if i.endswith(".nii.gz") and "_p" in i.lower()]
            if len(ct_)!=1 and len(seg_)!=1:
                print(len(ct_), len(seg_))
                continue
            ct = os.path.join(root_mrn_date, ct_[0])
            seg = os.path.join(root_mrn_date, seg_[0])
            #print(ct)
            #print(seg)
            ct_w = nib.load(ct)
            seg_w = nib.load(ct)
            # Check number of slices (assumes axial slices along third dimension)
            if ct_w.shape != seg_w.shape:
                print("Shape mismatch:", ct_w.shape, seg_w.shape, "→ Skipping", mrn, date)
                continue

            spacing_ct = ct_w.header.get_zooms()
            spacing_seg = seg_w.header.get_zooms()

            base_name = get_base_name(ct_[0])
            # Save data
            out_path = os.path.join(saving_path, base_name + ".npz")
            print("saving in ", out_path)
            np.savez(
                out_path,
                spacing_ct=spacing_ct,
                ct=ct_w.get_fdata(),  # native dtype
                seg=seg_w.get_fdata(),  # native dtype
                **info_dict
            )
            print("Saved:", out_path)
            print()

/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/1094315_D
['141125']
root_mrn_date:  /rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/1094315_D/141125
saving in  /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/1094315_141125__1.96__.npz


EOFError: Compressed file ended before the end-of-stream marker was reached

In [10]:

saving_path = "/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/"
import os
list_dir = [i.split("_")[0] for i in os.listdir(saving_path)]
d_dir = {}
for i in list_dir:
    if i in d_dir:
        d_dir[i] += 1
    else:
        d_dir[i] = 1



In [11]:
#csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/pyradiomics_sybil_demog_CV.csv"
csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/all_pyradiomics_sybil.csv"
import pandas as pd
df = pd.read_csv(csv_path)
df.shape

(623, 118)

In [12]:
d_df = {}
for i in df["MRN"].to_list():
    if i in d_df:
        d_df[i] += 1
    else:
        d_df[i] = 1
len(d_df)

122

In [13]:
def dict_diff(d1, d2):
    # keys in each
    k1 = set(d1.keys())
    k2 = set(d2.keys())

    added    = k2 - k1                # in d2 not in d1
    removed  = k1 - k2                # in d1 not in d2
    # keys present in both but with different values
    modified = {k: (d1[k], d2[k]) for k in k1 & k2 if d1[k] != d2[k]}
    # keys present in both and same value
    same     = {k for k in k1 & k2 if d1[k] == d2[k]}

    return added, removed, modified, same

# example
d1 = d_df
d2 = d_dir

added, removed, modified, same = dict_diff(d1, d2)
print("in dir not in df:   ", added)      # {'d'}
print("in df not in dir: ", removed)    # {'a'}
print("modified:", modified)   # {'c': (3, 4)}
print("same:    ", same)       # {'b'}


in dir not in df:    {'297791', '2338015'}
in df not in dir:  {'2338015B', '297791A'}
modified: {'1094315': (4, 3)}
same:     {'730556', '753033', '1151940', '2298483', '2179573', '939056', '2261315', '2317085', '754274', '937154', '1263077', '2391023', '845792', '2487906', '2166737', '831719', '823747', '2147359', '839203', '744086', '2246010', '2072002', '2343814', '872574', '1137934', '2109263', '748229', '802160', '2145117', '2281565', '838996', '281227', '1016339', '812470', '2432712', '422865', '477516', '2429727', '895053', '2521327', '944266', '1046669', '934625', '2388171', '311130', '2250825', '1093875', '2268709', '2389243', '1208921', '871833', '916855', '950270', '493464', '2079755', '619865', '2266872', '1105317', '2455935', '502083', '630484', '2424680', '743544', '2068766', '1093523', '2247360', '616175', '2280834', '2026140', '2247693', '1011153', '662995', '630766', '2321900', '420022', '2405062', '2413869', '842261', '809781', '2212191', '2280528', '2296044', '245468

In [24]:
df.shape

(623, 111)

In [1]:
[i for i in os.listdir(saving_path) if i.startswith("1094315")] #   422865

NameError: name 'os' is not defined